# Introduction to Artificial Intelligence - Homework Assignment 03 (20pts.)


* NETIDs:

This assignment covers the following topics:

* Principle Component Analysis
* KMeans Clustering
* Agglomerative Clustering
* Gaussian Mixture Models
* Support Vector Machines

and will consist of 5 tasks:

* Task 00: Load Dataset (0 pts.)
* Task 01: PCA Function
* Task 02: Clustering
    * Task 02-1: KMeans Clustering
    * Task 02-2: Agglomerative Clustering
    * Task 02-3: Gaussian Mixture Model Clustering
* Task 03: Clustering Method Evaluation
    * Task 03-1: IoU Metric
    * Model Evaluation
* Task 04: Support Vector Machines
    * Task 04-1: Feature vectors creation for SVM
    * Task 04-2: Support Vector Machines
    * Task 04-3: SVM Evaluation
* Task 05: Final Letter Data Extraction
    * Task 05-1: Computer Vision Region Cropping Function
    * Task 05-2: Letter Extraction

### *Story Progression*

The police need some help with those letters Detective Caulfield
delivered during class. The letters appear to be newspaper cutouts all
combined together into a single letter. The Police don't want to do all
the manual work of converting the letters into a text format and so
they've asked your your help!

As you try to fight off the spring break hangover, you realize you can
treat this as an image segmentation task. We need to figure out where in
the entire image each letter is. After hairing the dog, you have another
breakthrough idea, what if you just perform a clustering task with only
two clusters, foreground and background? Using the clustering methods we
saw in class, you decide to give it a shot!

## Task 00: Load Dataset
### Task 00: Description
#### Dataset Loading

We can use the imread function from the cv2 library to read our images
in. We should end up with the actual images shape being (595, 420, 3)
and the mask shape being (595, 420)

In [ ]:
import os
import cv2

import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.svm import SVC

# Set random seed for NumPy’s random number generator
#     Conceptually: makes random operations (e.g., random initialization in KMeans, shuffling data, or selecting random samples) produce the same results every time the code is run
#     aka your code becomes reporducable
np.random.seed(42)

try:
    import google.colab
    REPO_URL = "https://github.com/nd-cse-30124-fa25/cse-30124-homeworks.git"
    REPO_NAME = "cse-30124-homeworks"
    HW_FOLDER = "homework03" 

    # Clone repo if not already present
    if not os.path.exists(REPO_NAME):
        !git clone {REPO_URL}

    # cd into the homework folder
    %cd {REPO_NAME}/{HW_FOLDER}

except ImportError:
    pass

image_1_path = 'letters/note_page_1.png'
mask_1_path = 'letters/mask_page_1.png'

image_2_path = 'letters/note_page_2.png'
mask_2_path = 'letters/mask_page_2.png'

image_3_path = 'letters/note_page_3.png'
mask_3_path = 'letters/mask_page_3.png'

image_4_path = 'letters/note_page_4.png'
mask_4_path = 'letters/mask_page_4.png'


image_1 = cv2.imread(image_1_path)
image_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2RGB)
mask_1 = cv2.imread(mask_1_path, cv2.IMREAD_GRAYSCALE)

image_2 = cv2.imread(image_2_path)
image_2 = cv2.cvtColor(image_2, cv2.COLOR_BGR2RGB)
mask_2 = cv2.imread(mask_2_path, cv2.IMREAD_GRAYSCALE)

image_3 = cv2.imread(image_3_path)
image_3 = cv2.cvtColor(image_3, cv2.COLOR_BGR2RGB)
mask_3 = cv2.imread(mask_3_path, cv2.IMREAD_GRAYSCALE)

image_4 = cv2.imread(image_4_path)
image_4 = cv2.cvtColor(image_4, cv2.COLOR_BGR2RGB)
mask_4 = cv2.imread(mask_4_path, cv2.IMREAD_GRAYSCALE)

### *Story Progression*

You remember there were a number of clustering methods in class, but you
also remember that typically k-means clustering was the one used as a
sort of litmus test, so you figure that may as well be the place to
start

## Task 01: Principal Component Analysis
### Task 01: Description
#### PCA

*Add more instructions*

### Task 01: Code

In [ ]:
import numpy as np

class PCA:
    def __init__(self, n_components=None, whiten=False):
        """
        n_components:
          - None  -> keep all components
          - int   -> keep that many components
          - float in (0,1] -> keep the minimum # of components to reach this
                              fraction of explained variance
        whiten: if True, projected features are scaled to unit variance.
        """
        self.n_components = n_components
        self.whiten = whiten

    def fit(self, X):
        X = np.asarray(X, dtype=float)
        n_samples, n_features = X.shape

        # Center the data
        self.mean_ = X.mean(axis=0)
        Xc = X - self.mean_

        # SVD: Xc = U * diag(S) * Vt
        U, S, Vt = np.linalg.svd(Xc, full_matrices=False)

        # Variances along principal axes
        self.singular_values_ = S
        self.explained_variance_ = (S**2) / (n_samples - 1)
        total_var = self.explained_variance_.sum()
        self.explained_variance_ratio_ = self.explained_variance_ / total_var

        # Decide how many to keep
        self.n_components_ = self._resolve_n_components(X.shape[1])

        # Principal axes (each row is a component loading vector)
        self.components_ = Vt[:self.n_components_]
        return self

    def _resolve_n_components(self, n_features):
        if self.n_components is None:
            return min(n_features, self.singular_values_.size)
        if isinstance(self.n_components, int):
            k = int(self.n_components)
            if not (1 <= k <= min(n_features, self.singular_values_.size)):
                raise ValueError("n_components out of valid range.")
            return k
        if isinstance(self.n_components, float):
            thr = float(self.n_components)
            if not (0.0 < thr <= 1.0):
                raise ValueError("Float n_components must be in (0, 1].")
            cumsum = np.cumsum(self.explained_variance_ratio_)
            return int(np.searchsorted(cumsum, thr) + 1)
        raise TypeError("n_components must be None, int, or float in (0,1].")

    def transform(self, X):
        self._check_fitted()
        X = np.asarray(X, dtype=float)
        Xc = X - self.mean_
        Z = Xc @ self.components_.T
        if self.whiten:
            Z /= np.sqrt(self.explained_variance_[:self.n_components_] + 1e-12)
        return Z

    def inverse_transform(self, Z):
        self._check_fitted()
        Z = np.asarray(Z, dtype=float)
        if self.whiten:
            Z *= np.sqrt(self.explained_variance_[:self.n_components_] + 1e-12)
        Xhat = Z @ self.components_ + self.mean_
        return Xhat

    def fit_transform(self, X):
        return self.fit(X).transform(X)

    def _check_fitted(self):
        if not hasattr(self, "components_"):
            raise RuntimeError("Call fit(X) before transform/inverse_transform.")

# ---------------------------
# Tiny usage example
# ---------------------------
if __name__ == "__main__":
    rng = np.random.default_rng(0)
    # Make a 3D dataset with correlated features
    n = 400
    base = rng.normal(0, 1, size=(n, 1))
    x1 = base[:, 0] + 0.1 * rng.normal(size=n)
    x2 = 2 * base[:, 0] + 0.1 * rng.normal(size=n)
    x3 = rng.normal(0, 0.3, size=n)  # mostly noise
    X = np.c_[x1, x2, x3]

    pca = PCA(n_components=0.95)   # keep enough PCs to explain ≥95% variance
    Z = pca.fit_transform(X)

    print("n_components_:", pca.n_components_)
    print("Explained variance ratio:", pca.explained_variance_ratio_[:pca.n_components_])
    # Reconstruct to see the compression error
    X_recon = pca.inverse_transform(Z)
    recon_mse = np.mean((X - X_recon)**2)
    print("Reconstruction MSE:", recon_mse)

In [ ]:
def run_pca(x, n_components=2):
    pass


## Task 02: KMeans Clustering
### Task 02: Description
#### KMeans Clustering

*Add more instructions*

### Task 02: Code

In [ ]:
# -------------------------------------
# Task 1: Perform Kmeans Segmentation
# -------------------------------------
def kmeans_segmentation(image, pca_components=None):
    """
    Perform K-Means segmentation on an image

    Args:
        image: RGB image
        pca_components: Number of PCA components to use (None for no PCA)

    Returns:
        segmented_image: Image with pixel values replaced by cluster labels
    """
    # Reshape the image into a 2D array of pixels and 3 color values (RGB)
    pixels = image.reshape(-1, 3)

    # Apply PCA if specified
    if pca_components is not None:
        # TODO: Standardize pixel values to have zero mean and unit variance


        # TODO: Apply PCA to reduce dimensionality


        # TODO: Assign PCA-transformed data to X
        X = ...
    else:
        # If no PCA wanted, just set X = to the reshaped image
        X = ...

    # TODO: Perform K-Means clustering, fit Kmeans to data to get cluster labels for each pixel
    #    hint, we want to cluster the foreground and the background, so how many clusers should you use?

    # Reshape the labels back to the image shape
    segmented_image = labels.reshape(image.shape[:2])

    return segmented_image

## Task 03: Agglomerative Hierarchical Clustering
### Task 03: Description
#### Hierarchical Clustering

*Add more instructions*

#### *NOTE: This will likely crash your kernel, it's a pedagogical tool.*

### Task 03: Code

In [ ]:
# -------------------------------------
# Task 2: Perform Agglomerative Segmentation
# -------------------------------------
def agglomerative_segmentation(image, pca_components=None):
    """
    Perform Agglomerative Clustering on an image

    Args:
        image: RGB image
        pca_components: Number of PCA components to use (None for no PCA)

    Returns:
        segmented_image: Image with pixel values replaced by cluster labels
    """

    # Reshape the image into a 2D array of pixels and 3 color values (RGB)
    pixels = image.reshape(-1, 3)

    # Apply PCA if specified
    if pca_components is not None:
        # TODO: Standardize pixel values to have zero mean and unit variance


        # TODO: Apply PCA to reduce dimensionality


        # TODO: Assign PCA-transformed data to X
        X = ...
    else:
        X = ...

    # TODO: Apply Agglomerative Clustering


    # Reshape the labels back to the image shape
    segmented_image = labels.reshape(image.shape[:2])

    return segmented_image

### Task03: Short Answer Question

## Task 04: Gaussian Mixture Models
### Task 04: Description
#### GMMs

*Add more instructions*

### Task 04: Code

In [ ]:
# -------------------------------------
# Task 3: Perform Gaussian Mixture Model Segmentation
# -------------------------------------
def gmm_segmentation(image, pca_components=None):
    """
    Perform Gaussian Mixture Model segmentation on an image

    Args:
        image: RGB image
        pca_components: Number of PCA components to use (None for no PCA)

    Returns:
        segmented_image: Image with pixel values replaced by component labels
    """

    # Reshape the image into a 2D array of pixels and 3 color values (RGB)
    pixels = image.reshape(-1, 3)

    # Apply PCA if specified
    if pca_components is not None:
        # TODO: Standardize pixel values to have zero mean and unit variance


        # TODO: Apply PCA to reduce dimensionality


        # TODO: Assign PCA-transformed data to X
        X = ...
    else:
        X = ...

    # TODO: Apply GMM


    # Reshape the labels back to the image shape
    segmented_image = labels.reshape(image.shape[:2])

    return segmented_image

## Task 05: Clustering Evaluation
### Task 05-1: Description
#### Intersection over Union (IoU)

We'll need to evaluate how well our segmentation methods are doing. We
can use the intersection over union (IoU) metric to evaluate how well
our segmentation methods are doing.

Intersection over Union (IoU) is defined as the intersection of the
predicted mask and the ground truth mask divided by the union of the
predicted mask and the ground truth mask.

<span class="math display">\$\$IoU = \frac{TP}{TP + FP + FN}\$\$</span>

Where TP is the number of true positives, FP is the number of false
positives, and FN is the number of false negatives.

Really all this boils down to is looking at how close our predicted
labels, if we made every background cluster pixel black, and every
foreground cluster pixel 1, is to our ground-truth mask

In order to actually assess how well your models did, you of course need
something to compare against, so the police manually annotated 4 letters
and created masks for them

### Task 05-1: Code

In [ ]:
# -------------------------------------
# Task 6: Segmentation Metrics
# -------------------------------------
def calculate_segmentation_metrics(pred, truth, invert=False, visualize=False):
    """
    Calculate Intersection over Union (IoU) to measure segmentation accuracy

    Args:
        pred: Binary prediction mask (595, 420)
        truth: Binary ground truth mask (595, 420)

    Returns:
        iou: Intersection over Union (IoU) metric
    """
    if visualize:
      # Visualize the masks
      plt.figure(figsize=(10, 5))
      
      plt.subplot(1, 2, 1)
      plt.title('Prediction Mask')
      plt.imshow(pred, cmap='gray')
      plt.axis('off')
      
      plt.subplot(1, 2, 2)
      plt.title('Ground Truth Mask')
      plt.imshow(truth, cmap='gray')
      plt.axis('off')
      
      plt.show()


    # TODO: Convert pred to boolean mask


    # TODO: If invert is true, then flip foreground and background labels
    if invert:
      pass

    # TODO: Convert truth to boolean mask


    # TODO: Calculate intersection and union


    # TODO: calculate IoU (Intersection over Union)
    #    hint: this is also known as the Jaccard index
    iou = ...
    print('IoU:', iou)

    return iou

### Task05-2: Description
#### Evaluation of Clustering Methods

We should probably explore how well each of our unsupervised
segmentation methods worked, with different numbers of PCA components.

### Task05-2: Code

In [ ]:
print("\nPerforming Unsupervised Segmentation...")
for pca_components in [None, 1, 2]:
    # TODO: Run Kmeans segmentation, evaluate how well it did
    print(f"Running KMeans with PCA components: {pca_components}")
    segmented_image_kmeans = ...


    # TODO Run GMM, evaluate how well it did
    print(f"Running Gaussian Mixture Model with PCA components: {pca_components}")
    segmented_image_gmm = ...


### Task05-2: Expected Output

    Performing Unsupervised Segmentation...
    Running KMeans with PCA components: None
    IoU: 0.8150511446663399
    Running Gaussian Mixture Model with PCA components: None
    IoU: 0.8456018086408212
    Running KMeans with PCA components: 1
    IoU: 0.7816316084529681
    Running Gaussian Mixture Model with PCA components: 1
    IoU: 0.8448758633563543
    Running KMeans with PCA components: 2
    IoU: 0.7847515051063914
    Running Gaussian Mixture Model with PCA components: 2
    IoU: 0.8456018086408212

*As we talked about in class, kmeans can be non-deterministic so please
don't fret if your numbers are a little different*

### *Story Progression*

The unsupervised segmentation seems to work pretty well, but you wonder
if a supervised method might work even better. We could probably set
this up in a similar way to the clustering task, but instead of using
two clusters (foreground and background), we can have two classes
(foreground and background).

Luckily the police have manually created a binary mask for us that we
can use as labelled data to train our supervised model.

Unluckily, we need to somehow let each pixel know about the surrounding
pixels. So we'll need to somehow create a feature vector for each pixel
that contains information about the pixel and the surrounding pixels.

Maybe we can take a patch around each pixel and use the RGB values of
the pixels in the patch as features?

## Task06: Supervised Image Segmentation
### Task06-1: Description
#### Feature Creation

*More Here*

### Task06-1: Code

In [ ]:
# -------------------------------------
# Task 4: Write Extract Features Function
# -------------------------------------
def extract_features(image, patch_size=5):
    """
    Extract features for each pixel using RGB values and local neighborhood.

    Args:
        image: RGB image (595, 420, 3)
        patch_size: Size of the local neighborhood patch (must be odd)

    Returns:
        features: Array of shape (n_pixels, n_features) (249900, 78) (595 x 420, 5 x 5 x 3 + 1 x 3)
    """
    height, width = ...

    # TODO: Add padding to image
    padding = ...

    padded = cv2.copyMakeBorder(
        image,
        padding, padding, padding, padding,
        cv2.BORDER_REFLECT
    )


    # TODO: Calculate total number of pixels
    n_pixels = ...

    # TODO: Calculate total number of features per pixel
    n_features = ...

    # TODO: Initialize data structue to store features for all pixels
    features = ...

    # TODO: Loop through pixels and extracts features for each pixel
    #     hint: the feature vector will be all RGB values for every surrounding pixel concatenated with the pixel itself
    #     hint: this results in a 78 dimensional feature vector, for each pixel!
    #     hint: 5 x 5 path = 25, 3 colors per pixel gives 3 x 25 = 75 + 3 for the pixel itself again gives 78
    #     hint: use current pixel RGB values and the surrounding local patch


    # Return feature matrix 
    return features

### Task 06-2: Description
#### Supervised Image Segmentation via SVM

*Instructions*

### Task 06-2: Code

In [ ]:
# -------------------------------------
# Task 5: SVM Segmentation
# -------------------------------------
def svm_segmentation(train_image, train_mask, test_image, kernel='rbf', pca_components=10):
    # TODO: Extract features of training image
    X = ...
    y = (train_mask > 0).reshape(-1)

    # TODO: Scale features


    # TODO: Apply PCA


    # TODO: Train SVM


    # TODO: Extract features of testing image


    # TODO: Apply transformations to testing image
    #     hint: transformations means scaling and dimensionality reduction


    # TODO: Predict on test image


    # Reshape predictions to match image matrix shape
    segmented_image = predictions.reshape(test_image.shape[:2])

    return segmented_image

### Task 06-3: Description
#### SVM Segmentation Evaluation

Now we can try our supervised segmentation method (SVM) with different
kernels and different numbers of PCA components.

*This will take a while to run, so be patient!*

I encourage you to try different amounts for PCA to see if you can beat
my results, but the three I tested were 1, 5, and 10

### Task 06-3: Code

In [ ]:
print("\nPerforming Supervised Segmentation...")
for kernel in ['linear', 'poly', 'rbf']:       # Conceptual question: what do 'linear', 'poly' and 'rbf' stand for? Why are we trying each of them as our kernel variable?
    for pca_components in [1, 5, 10]:
        # TODO: SVM segmentation and evaluation
        print(f"Running SVM with kernel: {kernel} and PCA components: {pca_components}")
        segmented_image_svm = ...

### Task 06-3: Expected Output

    Performing Supervised Segmentation...
    Running SVM with kernel: linear and PCA components: 1
    IoU: 0.7939733382706887
    Running SVM with kernel: linear and PCA components: 5
    IoU: 0.7906213147732772
    Running SVM with kernel: linear and PCA components: 10
    IoU: 0.8016820011500843
    Running SVM with kernel: poly and PCA components: 1
    IoU: 0.7570233911041068
    Running SVM with kernel: poly and PCA components: 5
    IoU: 0.8301359011807786
    Running SVM with kernel: poly and PCA components: 10
    IoU: 0.8775227199564151
    Running SVM with kernel: rbf and PCA components: 1
    IoU: 0.813663710080135
    Running SVM with kernel: rbf and PCA components: 5
    IoU: 0.914427174421753
    Running SVM with kernel: rbf and PCA components: 10
    IoU: 0.9516093009405142

## Task 07: Final Letter Region Extraction
### Task 07-1: Description
#### Region detection and localization function

This computer vision stuff is outside the scope of this course, so the
code is just provided for you. But this is how we can extract the region
of each letter using our segmentation!

### Task 07-1: Code

In [ ]:
from skimage.measure import label, regionprops
import os

def extract_and_save_letters(segmented_image, original_image, output_dir='letters'):
    """
    Extract each segmented letter and save as a separate PNG file.
    
    Args:
        segmented_image: Segmented image with unique labels for each letter
        original_image: Original RGB image
        output_dir: Directory to save the extracted letter images
    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Label connected components
    labeled_image = label(segmented_image)
    
    # Iterate over each labeled region
    for region in regionprops(labeled_image):
        # Extract the bounding box of the region
        min_row, min_col, max_row, max_col = region.bbox
        
        # Extract the region from the original image
        letter_image = original_image[min_row:max_row, min_col:max_col]
        
        # Save the extracted letter as a PNG file
        letter_filename = os.path.join(output_dir, f'letter_{region.label}.png')
        cv2.imwrite(letter_filename, cv2.cvtColor(letter_image, cv2.COLOR_RGB2BGR))

### Task 07-2: Description
#### Letter extraction

Lets actually use our best combination to extract each letter as a png!

### Task 07-2: Code

In [ ]:
segmented_image = svm_segmentation(train_image, train_mask, test_image, kernel='rbf', pca_components=10)
extract_and_save_letters(segmented_image, test_image)

### *Story Progression*

Now that we have all of the letters extracted, we can perform Optical
Character Recognition (OCR) on each letter to extract the text. The next
homework we'll explore how to do this with a basic Feed-Forward Neural
Network (which you'll be writing from scratch) and a Convolutional
Neural Network (which you'll be using from PyTorch).